# Installs

In [ ]:
# Kaggle API for downloading the datasets
!pip install --upgrade -q kaggle

# Optuna for parameter search
!pip install -q optuna

!pip install --upgrade xgboost

# upgrask sklearn
!pip install --upgrade scikit-learn

!pip install category_encoders
!pip install catboost
# lighgbm gpu compatible
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

     |████████████████████████████████| 296kB 7.9MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 174kB 27.0MB/s 
     |████████████████████████████████| 143kB 36.2MB/s 
     |████████████████████████████████| 112kB 25.0MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 166.7MB 41kB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
     |████████████████████████████████| 22.3MB 1.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 67.3MB 64kB/s 
Cloning into 'LightGBM'...
remote: Enumerating objects: 22151, done.
remote: Counting objects: 100% (697/697), done.


# Imports

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_predict,cross_val_score,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgbm
import xgboost as xgb
import pickle
from google.colab import files
import category_encoders as ce
from catboost import CatBoostClassifier, Pool
from sklearn.feature_selection import SelectKBest, f_classif,chi2,SelectPercentile
from scipy.stats import rankdata
from sklearn.mixture import GaussianMixture 
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve

import string
import re

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



# Load Data

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# get a list of kaggle competitions
#!kaggle competitions list

# download the target competition
!kaggle competitions download 'tabular-playground-series-may-2021'

# unzip the files if needed
## !unzip tabular-playground-series-jan-2021.zip

# load the data
df_all = pd.read_csv("train.csv.zip")
df_test = pd.read_csv("test.csv.zip")
df_submission = pd.read_csv("sample_submission.csv.zip")

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes
  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 56.6MB/s]
  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 118MB/s]
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 132MB/s]


# Data Analysis

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 52 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          100000 non-null  int64 
 1   feature_0   100000 non-null  int64 
 2   feature_1   100000 non-null  int64 
 3   feature_2   100000 non-null  int64 
 4   feature_3   100000 non-null  int64 
 5   feature_4   100000 non-null  int64 
 6   feature_5   100000 non-null  int64 
 7   feature_6   100000 non-null  int64 
 8   feature_7   100000 non-null  int64 
 9   feature_8   100000 non-null  int64 
 10  feature_9   100000 non-null  int64 
 11  feature_10  100000 non-null  int64 
 12  feature_11  100000 non-null  int64 
 13  feature_12  100000 non-null  int64 
 14  feature_13  100000 non-null  int64 
 15  feature_14  100000 non-null  int64 
 16  feature_15  100000 non-null  int64 
 17  feature_16  100000 non-null  int64 
 18  feature_17  100000 non-null  int64 
 19  feature_18  100000 non-n

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 51 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id          50000 non-null  int64
 1   feature_0   50000 non-null  int64
 2   feature_1   50000 non-null  int64
 3   feature_2   50000 non-null  int64
 4   feature_3   50000 non-null  int64
 5   feature_4   50000 non-null  int64
 6   feature_5   50000 non-null  int64
 7   feature_6   50000 non-null  int64
 8   feature_7   50000 non-null  int64
 9   feature_8   50000 non-null  int64
 10  feature_9   50000 non-null  int64
 11  feature_10  50000 non-null  int64
 12  feature_11  50000 non-null  int64
 13  feature_12  50000 non-null  int64
 14  feature_13  50000 non-null  int64
 15  feature_14  50000 non-null  int64
 16  feature_15  50000 non-null  int64
 17  feature_16  50000 non-null  int64
 18  feature_17  50000 non-null  int64
 19  feature_18  50000 non-null  int64
 20  feature_19  50000 non-null  

In [ ]:
df_all['target']

0        Class_2
1        Class_1
2        Class_1
3        Class_4
4        Class_2
          ...   
99995    Class_1
99996    Class_2
99997    Class_3
99998    Class_2
99999    Class_3
Name: target, Length: 100000, dtype: object

In [ ]:
df_all['target'].value_counts()/len(df_all)*100

Class_2    57.497
Class_3    21.420
Class_4    12.593
Class_1     8.490
Name: target, dtype: float64

In [ ]:
for col in list(df_all.columns):
  if col.startswith("f"):
    print(f"col: {col},nunique:{df_all[col].nunique()},min:{df_all[col].min()},max: {df_all[col].max()}")

col: feature_0,nunique:11,min:0,max: 10
col: feature_1,nunique:31,min:0,max: 31
col: feature_2,nunique:7,min:0,max: 6
col: feature_3,nunique:26,min:0,max: 26
col: feature_4,nunique:38,min:0,max: 38
col: feature_5,nunique:11,min:0,max: 10
col: feature_6,nunique:28,min:0,max: 27
col: feature_7,nunique:32,min:0,max: 31
col: feature_8,nunique:37,min:0,max: 39
col: feature_9,nunique:18,min:0,max: 17
col: feature_10,nunique:17,min:0,max: 16
col: feature_11,nunique:13,min:0,max: 12
col: feature_12,nunique:12,min:0,max: 11
col: feature_13,nunique:4,min:0,max: 3
col: feature_14,nunique:52,min:0,max: 51
col: feature_15,nunique:21,min:0,max: 20
col: feature_16,nunique:20,min:0,max: 19
col: feature_17,nunique:17,min:0,max: 16
col: feature_18,nunique:14,min:0,max: 13
col: feature_19,nunique:58,min:-2,max: 55
col: feature_20,nunique:18,min:0,max: 17
col: feature_21,nunique:36,min:0,max: 36
col: feature_22,nunique:5,min:0,max: 4
col: feature_23,nunique:20,min:0,max: 19
col: feature_24,nunique:35,min:

# Logistic Regression



In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

In [ ]:
encoder = OneHotEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
#X = all_encoded[0:len(X)]
#X_test = all_encoded[len(X):]
X = all_encoded.tocsr()[0:len(X)]
X_test = all_encoded [len(df_all):]

In [ ]:
df_all['target'].value_counts()

Class_2    57497
Class_3    21420
Class_4    12593
Class_1     8490
Name: target, dtype: int64

In [ ]:
params = {
          'penalty':'l2',
          'multi_class':'ovr',
          'solver':'lbfgs',  
          'C':0.01,
          'max_iter':10000 ,
          'class_weight':None        
          }

In [ ]:
name = 'logistic_regression'
k=5
seed_list=[0,1,2]
kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
oof = np.zeros((len(df_all),4))
test_preds_list = []
score_list = []
fold=1
  
splits = list(kf.split(X,y))
fold = 1
for train_idx, val_idx in splits:
  X_train, X_val = X[train_idx], X[val_idx]
  y_train, y_val = y[train_idx], y[val_idx]

  val_preds_list = []

  for seed in seed_list:
    
    # fit and run model
    
    base_model = LogisticRegression(**params,random_state=seed)
    model = CalibratedClassifierCV(base_model, method='sigmoid', cv=k)


    model.fit(X_train,y=y_train)

    
    val_preds_list.append(model.predict_proba(X_val))
    test_preds_list.append(model.predict_proba(X_test))
    
  oof[val_idx] = np.mean(val_preds_list,axis=0)
  score = log_loss(y_val, oof[val_idx])
  print(f"fold: {fold},log_loss: {score}")
  score_list.append(score)
  # print(f"fold: {fold}, class0 tr %: {y_train.value_counts()[0]/len(y_train)}, class0 val %: {y_val.value_counts()[0]/len(y_val)} ")
  fold +=1
  
cv_logloss = np.mean(score_list)
print(f"{name} ,log_loss: {cv_logloss}")

preds= np.mean(test_preds_list,axis=0)


file_name_oof = "logistic_3seeds_oof.txt"
file_name_test = "logistic_3seeds_test.csv"
with open(file_name_oof, "wb") as fp:
      pickle.dump(oof, fp)

files.download(file_name_oof)

df_submission[['Class_1','Class_2','Class_3','Class_4']] = preds
df_submission.to_csv(file_name_test,index=None)
files.download(file_name_test) 

fold: 1,log_loss: 1.0896463404941772
fold: 2,log_loss: 1.0942270580398394
fold: 3,log_loss: 1.0891577719590186
fold: 4,log_loss: 1.0909161253698327
fold: 5,log_loss: 1.0913208228386768
logistic_regression ,log_loss: 1.091053623740309


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Logistic Regression Tuning

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

In [ ]:
encoder = OneHotEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
#X = all_encoded[0:len(X)]
#X_test = all_encoded[len(X):]
X = all_encoded.tocsr()[0:len(X)]
X_test = all_encoded [len(df_all):]

In [ ]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial,data=X,target=y):
  """
  """
  param_space = {
          'C': trial.suggest_loguniform('C', 1e-3, 1e2),
          'penalty':'l2',
          'solver': trial.suggest_categorical('solver',['newton-cg', 'lbfgs', 'sag', 'saga']),
          'multi_class':trial.suggest_categorical('multi_class',['ovr','multinomial']),
          'max_iter':10000,
          'class_weight':trial.suggest_categorical('class_weight',['balanced',None])  ,
           'n_jobs':-1
                }
            
  #X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.1,random_state=2021,stratify=y)
  k=5
  seed_list=[0]
  kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
  oof = np.zeros((len(df_all),4))
  score_list = []
  fold=1
  
  splits = list(kf.split(X,y))
  for train_idx, val_idx in splits:
    X_train, X_val = X[train_idx,:], X[val_idx,:]
    y_train, y_val = y[train_idx], y[val_idx]
  
    val_preds_list = []
  
    for seed in seed_list:
      # fit and run model
      param_space['random_state'] = seed

      model = LogisticRegression(**param_space)
      #model = CalibratedClassifierCV(base_model, method='sigmoid', cv=k)
      model.fit(X_train,y=y_train)

    
      val_preds_list.append(model.predict_proba(X_val))
     #test_preds_list.append(model.predict_proba(X_test)[:,1])
    
    oof[val_idx] = np.mean(val_preds_list,axis=0)
    score = log_loss(y_val, oof[val_idx])
    print(f"fold: {fold},logloss: {score}")
    score_list.append(score)
    fold +=1
  
  cv_logloss = np.mean(score_list)
  
  return cv_logloss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-05-23 21:34:19,970] A new study created in memory with name: no-name-e54f9865-05d6-4133-b5ea-f97b94246e0c


fold: 1,logloss: 1.3764405615079547
fold: 2,logloss: 1.3834771132244839
fold: 3,logloss: 1.368472549230804
fold: 4,logloss: 1.3779910953879444


[I 2021-05-23 21:50:43,588] Trial 0 finished with value: 1.3767282841741353 and parameters: {'C': 25.59567454799754, 'solver': 'lbfgs', 'multi_class': 'multinomial', 'class_weight': 'balanced'}. Best is trial 0 with value: 1.3767282841741353.


fold: 5,logloss: 1.37726010151949
fold: 1,logloss: 1.0900952690611299
fold: 2,logloss: 1.0948244337170032
fold: 3,logloss: 1.0893037142024011
fold: 4,logloss: 1.0911801388813611


[I 2021-05-23 21:51:08,441] Trial 1 finished with value: 1.0914300977030766 and parameters: {'C': 0.011306868379481878, 'solver': 'lbfgs', 'multi_class': 'ovr', 'class_weight': None}. Best is trial 1 with value: 1.0914300977030766.


...

[I 2021-05-23 22:18:48,191] Trial 4 finished with value: 1.3549259519169254 and parameters: {'C': 0.005769650714001403, 'solver': 'saga', 'multi_class': 'multinomial', 'class_weight': 'balanced'}. Best is trial 1 with value: 1.0914300977030766.


fold: 5,logloss: 1.3551030534357023
fold: 1,logloss: 1.3502231290723012
fold: 2,logloss: 1.3532840598910092
fold: 3,logloss: 1.3498243592864143
fold: 4,logloss: 1.351104003195148


[I 2021-05-23 22:19:26,663] Trial 5 finished with value: 1.3512587421478046 and parameters: {'C': 0.028666703876421454, 'solver': 'newton-cg', 'multi_class': 'ovr', 'class_weight': 'balanced'}. Best is trial 1 with value: 1.0914300977030766.


fold: 5,logloss: 1.3518581592941503
fold: 1,logloss: 1.3792497069330918
fold: 2,logloss: 1.3871583635506648
fold: 3,logloss: 1.3712495107058411
fold: 4,logloss: 1.3810230821585758


KeyboardInterrupt: ignored

# Random Forest

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [ ]:
params = {
          'bootstrap':True,
          'max_depth':30,
          'max_features':'auto'  ,
          'min_samples_leaf' :10,
          'min_samples_split':5,
          'n_estimators':1560
          }

In [ ]:
name = 'random_forest'
k=5
seed_list=[0,1,2]
kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
oof = np.zeros((len(df_all),4))
test_preds_list = []
score_list = []
fold=1
  
splits = list(kf.split(X,y))
fold = 1
for train_idx, val_idx in splits:
  X_train, X_val = X[train_idx], X[val_idx]
  y_train, y_val = y[train_idx], y[val_idx]

  val_preds_list = []

  for seed in seed_list:
    
    # fit and run model
    
    base_model = RandomForestClassifier(**params,random_state=seed)
    model = CalibratedClassifierCV(base_model, method='sigmoid', cv=k)

    model.fit(X_train,y=y_train)

    
    val_preds_list.append(model.predict_proba(X_val))
    test_preds_list.append(model.predict_proba(X_test))
    
  oof[val_idx] = np.mean(val_preds_list,axis=0)
  score = log_loss(y_val, oof[val_idx])
  print(f"fold: {fold},log_loss: {score}")
  score_list.append(score)
  # print(f"fold: {fold}, class0 tr %: {y_train.value_counts()[0]/len(y_train)}, class0 val %: {y_val.value_counts()[0]/len(y_val)} ")
  fold +=1
  
cv_logloss = np.mean(score_list)
print(f"{name} ,log_loss: {cv_logloss}")

preds= np.mean(test_preds_list,axis=0)


file_name_oof = "rfc_3seeds_oof.txt"
file_name_test = "rfc_3seeds_test.csv"
with open(file_name_oof, "wb") as fp:
      pickle.dump(oof, fp)

#files.download(file_name_oof)

df_submission[['Class_1','Class_2','Class_3','Class_4']] = preds
df_submission.to_csv(file_name_test,index=None)
#files.download(file_name_test) 

fold: 1,log_loss: 1.096954666028222
fold: 2,log_loss: 1.099633708005841
fold: 3,log_loss: 1.0972724875462816
fold: 4,log_loss: 1.0970865994352583
fold: 5,log_loss: 1.0974270829667088
random_forest ,log_loss: 1.0976749087964623


# Random Forest Tuning

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [ ]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial,data=X,target=y):
  """
  """
  param_space = {
               'max_depth': trial.suggest_int('max_depth', 2, 30),
               'n_estimators': trial.suggest_int('n_estimators', 200,2000,10),
               'max_features': trial.suggest_categorical('max_features',['auto','sqrt']),
               'min_samples_split':trial.suggest_categorical('min_samples_split',[2,5,10]),
               'bootstrap' : trial.suggest_categorical('bootstrap',[True,False]),
               'min_samples_leaf':trial.suggest_categorical('min_samples_leaf',[2,5,10])

                }
            
  model = RandomForestClassifier(**param_space,random_state=random_seed,n_jobs=-1)
  kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_seed)
  scores = cross_val_score(model,X,y,scoring='neg_log_loss',cv=kf)
  cv_score = -1*scores.mean()
      
  return cv_score

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 25)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-05-20 19:06:59,333] A new study created in memory with name: no-name-428bead7-59a4-4895-934b-a0f3a7f79456
[I 2021-05-20 19:14:07,483] Trial 0 finished with value: 1.1037324397093398 and parameters: {'max_depth': 30, 'n_estimators': 990, 'max_features': 'sqrt', 'min_samples_split': 10, 'bootstrap': False, 'min_samples_leaf': 2}. Best is trial 0 with value: 1.1037324397093398.
[I 2021-05-20 19:22:15,105] Trial 1 finished with value: 1.1011955293418823 and parameters: {'max_depth': 24, 'n_estimators': 1890, 'max_features': 'auto', 'min_samples_split': 5, 'bootstrap': True, 'min_samples_leaf': 5}. Best is trial 1 with value: 1.1011955293418823.
[I 2021-05-20 19:25:17,256] Trial 2 finished with value: 1.1041724861895943 and parameters: {'max_depth': 30, 'n_estimators': 420, 'max_features': 'auto', 'min_samples_split': 10, 'bootstrap': False, 'min_samples_leaf': 2}. Best is trial 1 with value: 1.1011955293418823.
[I 2021-05-20 19:26:51,829] Trial 3 finished with value: 1.114695508750

Number of finished trials: 25
Best trial: {'max_depth': 30, 'n_estimators': 1560, 'max_features': 'auto', 'min_samples_split': 5, 'bootstrap': True, 'min_samples_leaf': 10}


In [ ]:
study.best_params

{'bootstrap': True,
 'max_depth': 30,
 'max_features': 'auto',
 'min_samples_leaf': 10,
 'min_samples_split': 5,
 'n_estimators': 1560}

# LightGBM

In [ ]:
# X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
# y = df_all['target'].values
# X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
# random_seed = 0

# encoder = OrdinalEncoder()
# all_encoded = encoder.fit_transform(X.append(X_test))
# X = all_encoded[0:len(X)]
# X_test = all_encoded[len(X):]

# cat_features = np.arange(0,X.shape[1]).tolist()

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [ ]:
# params = {
#  'colsample_bytree': .1,
#  'learning_rate': .01,
#  'max_depth': 3,
#  'min_child_samples': 81,
#  'min_child_weight':0.5261052720119306,

#  'num_leaves': 109,
#  'reg_alpha': 14.411664782568891,
#  'reg_lambda': 0.011351864739530503,
#  'subsample': 0.04015184467061349
#  }


params = {
  'learning_rate': .02,
   'max_depth': 3,
    'num_leaves': 6,
    'min_split_gain': 0.17865452483871047,
    'reg_alpha': 9.540720621520459,
    'reg_lambda': 4.5781292529661375,
    'colsample_bytree': 0.0644950794287173,
    'subsample': 0.9314592865852914,
    'subsample_freq': 7,
    'min_child_samples': 57

#  'colsample_bytree': .1,
#  'max_depth': 3,
#  'min_child_samples': 81,
#  'min_child_weight':0.5261052720119306,

#  'num_leaves': 109,
#  'reg_alpha': 14.411664782568891,
#  'reg_lambda': 0.011351864739530503,
#  'subsample': 0.04015184467061349
 }

In [ ]:
params_lgbm = params
params_lgbm['boosting_type'] = 'gbdt'
params_lgbm['device'] = 'gpu'
params_lgbm ['objective'] = 'multiclasss'
params_lgbm ['num_classes'] = 4,

params_lgbm ['metric'] = 'multi_logloss'
params_lgbm ['verbosity'] = -1
params_lgbm ['n_estimators']= 50000
#params_lgbm["cat_feature"] = cat_features

name = 'lighgbm_3seeds_5fold'
k=5
seed_list=[0,1,2]
kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
oof = np.zeros((len(df_all),4))
test_preds_list = []
score_list = []
fold=1
  
splits = list(kf.split(X,y))
fold = 1
for train_idx, val_idx in splits:
  X_train, X_val = X[train_idx], X[val_idx]
  y_train, y_val = y[train_idx], y[val_idx]

  val_preds_list = []

  for seed in seed_list:
    
    # fit and run model
    params_lgbm['random_state'] = seed
    
    model = lgbm.LGBMClassifier(**params_lgbm)
    
    model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val,y_val)],
              early_stopping_rounds=100,
              eval_names=['train','val'],verbose=200)

    
    val_preds_list.append(model.predict_proba(X_val))
    test_preds_list.append(model.predict_proba(X_test))
    
  oof[val_idx] = np.mean(val_preds_list,axis=0)
  score = log_loss(y_val, oof[val_idx])
  print(f"fold: {fold},log_loss: {score}")
  score_list.append(score)
  # print(f"fold: {fold}, class0 tr %: {y_train.value_counts()[0]/len(y_train)}, class0 val %: {y_val.value_counts()[0]/len(y_val)} ")
  fold +=1
  
cv_logloss = np.mean(score_list)
print(f"{name} ,log_loss: {cv_logloss}")

preds= np.mean(test_preds_list,axis=0)


file_name_oof = name +"_oof.txt"
file_name_test = name + "_test.csv"
with open(file_name_oof, "wb") as fp:
      pickle.dump(oof, fp)

#files.download(file_name_oof)

df_submission[['Class_1','Class_2','Class_3','Class_4']] = preds
df_submission.to_csv(file_name_test,index=None)
#files.download(file_name_test) 

Training until validation scores don't improve for 100 rounds
[200]	train's multi_logloss: 1.10559	val's multi_logloss: 1.10679
[400]	train's multi_logloss: 1.09827	val's multi_logloss: 1.10052
[600]	train's multi_logloss: 1.09365	val's multi_logloss: 1.09686
[800]	train's multi_logloss: 1.09028	val's multi_logloss: 1.09443
[1000]	train's multi_logloss: 1.08782	val's multi_logloss: 1.09282
[1200]	train's multi_logloss: 1.08592	val's multi_logloss: 1.09169
[1400]	train's multi_logloss: 1.08445	val's multi_logloss: 1.09087
[1600]	train's multi_logloss: 1.08327	val's multi_logloss: 1.09034
[1800]	train's multi_logloss: 1.08231	val's multi_logloss: 1.0899
[2000]	train's multi_logloss: 1.08154	val's multi_logloss: 1.08961
[2200]	train's multi_logloss: 1.08088	val's multi_logloss: 1.08938
[2400]	train's multi_logloss: 1.08032	val's multi_logloss: 1.08924
[2600]	train's multi_logloss: 1.07983	val's multi_logloss: 1.08914
[2800]	train's multi_logloss: 1.07946	val's multi_logloss: 1.08904
[3000

# Lightgbm Tuning

In [ ]:
# X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
# y = df_all['target'].values
# X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
# random_seed = 0

# encoder = OrdinalEncoder()
# all_encoded = encoder.fit_transform(X.append(X_test))
# X = all_encoded[0:len(X)]
# X_test = all_encoded[len(X):]

# cat_features = np.arange(0,X.shape[1]).tolist()

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [ ]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial,data=X,target=y):
  """
  """
  param_space = {
               'device':'gpu',  # Use GPU acceleration
               'boosting_type': 'gbdt',
               'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
               'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
               'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1 , 1.0),
               'subsample': trial.suggest_float('subsample', 0.1, 1.0),
               'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 1e-2),
               'num_leaves': trial.suggest_int("num_leaves", 31,256),
               'min_child_samples': trial.suggest_int('min_child_samples', 1, 500),
               'max_depth':trial.suggest_int('max_depth',3,127),
               'n_estimators':100000,
               'objective':'multiclass',
               'metric':'multi_logloss'
                }
            
  #X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.1,random_state=2021,stratify=y)
  k=5
  seed_list=[0]
  kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
  oof = np.zeros((len(df_all),4))
  score_list = []
  fold=1
  
  splits = list(kf.split(X,y))
  for train_idx, val_idx in splits:
    X_train, X_val = X[train_idx,:], X[val_idx,:]
    y_train, y_val = y[train_idx], y[val_idx]
  
    val_preds_list = []
  
    for seed in seed_list:
      # fit and run model
      param_space['random_state'] = seed

      model = lgbm.LGBMClassifier(**param_space)
      model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val,y_val)],
                early_stopping_rounds=100,
                eval_names=['train','val'],verbose=0)
    
      val_preds_list.append(model.predict_proba(X_val))
     #test_preds_list.append(model.predict_proba(X_test)[:,1])
    
    oof[val_idx] = np.mean(val_preds_list,axis=0)
    score = log_loss(y_val, oof[val_idx])
    print(f"fold: {fold},logloss: {score}")
    score_list.append(score)
    fold +=1
  
  cv_logloss = np.mean(score_list)
  
  return cv_logloss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-05-23 19:46:25,041] A new study created in memory with name: no-name-fb79f113-3753-4660-a29d-2bb115b0a193


fold: 1,logloss: 1.1198736476631492


KeyboardInterrupt: ignored

In [ ]:
study.best_params

{'cat_l2': 1,
 'cat_smooth': 41,
 'colsample_bytree': 0.10152839180930642,
 'learning_rate': 0.028561805912614473,
 'max_depth': 77,
 'min_child_samples': 256,
 'min_data_per_group': 293,
 'num_leaves': 69,
 'reg_alpha': 41.076503175793036,
 'reg_lambda': 0.0011579102445110068,
 'subsample': 0.3473382674747556}

# XGBoost

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].map({"Class_1":0,"Class_2":1,"Class_3":2,"Class_4":3}).values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [ ]:
# params = { 
# 'max_depth': 6,  
# 'learning_rate': 0.1,
# 'gamma': 0.5, 
# 'reg_lambda': 20, 
# 'reg_alpha': 10, 
# 'colsample_bytree': 0.1
#               }

In [ ]:
params =  {'lambda': 1.3718620937297796, 
           'alpha': 6.395781966352342, 
           'colsample_bytree': 0.2390564723786096, 
           'colsample_bynode': 0.7459555518737353, 
           'colsample_bylevel': 0.36002014547566097, 
           'subsample': 0.6302863949739616,
           'eta': 0.01, 
           'grow_policy': 'lossguide', 
           'max_depth': 19, 
           'min_child_weight': 28, 
           'max_bin': 258, 
           'deterministic_histogram': False}

In [ ]:
params_xgb = params
params_xgb["tree_method"] = "gpu_hist"
params_xgb["predictor"] = 'gpu_predictor'
params_xgb["objective"] = 'multi:softprob'
params_xgb["num_class"] = 4
#params_xgb["eval_metric"] ='logloss'

name = 'xgboost_3seeds_5fold'
k=5
seed_list=[0,1,2]
kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
oof = np.zeros((len(df_all),4))
test_preds_list = []
score_list = []
fold=1
  
splits = list(kf.split(X,y))
fold = 1
for train_idx, val_idx in splits:
  X_train, X_val = X[train_idx], X[val_idx]
  y_train, y_val = y[train_idx], y[val_idx]

  val_preds_list = []

  for seed in seed_list:
    
    # fit and run model
    params_xgb['seed'] = seed
    
    dtrain = xgb.DMatrix(data=X_train, label=y_train)
    dval = xgb.DMatrix(data=X_val, label=y_val)
    dtest = xgb.DMatrix(data=X_test)
    
    model = xgb.train(params_xgb, dtrain,\
                       evals=[(dtrain,'train'),(dval,'val')],\
                       verbose_eval=100,
                       early_stopping_rounds=100,
                       num_boost_round=100000)
    
    

    
    val_preds_list.append(model.predict(dval))
    test_preds_list.append(model.predict(dtest))
    
  oof[val_idx] = np.mean(val_preds_list,axis=0)
  score = log_loss(y_val, oof[val_idx])
  print(f"fold: {fold},log_loss: {score}")
  score_list.append(score)
  # print(f"fold: {fold}, class0 tr %: {y_train.value_counts()[0]/len(y_train)}, class0 val %: {y_val.value_counts()[0]/len(y_val)} ")
  fold +=1
  
cv_logloss = np.mean(score_list)
print(f"{name} ,log_loss: {cv_logloss}")

preds= np.mean(test_preds_list,axis=0)


file_name_oof = name + "_oof.txt"
file_name_test = name + "_test.csv"
with open(file_name_oof, "wb") as fp:
      pickle.dump(oof, fp)

files.download(file_name_oof)

df_submission[['Class_1','Class_2','Class_3','Class_4']] = preds
df_submission.to_csv(file_name_test,index=None)
files.download(file_name_test) 

[17:18:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-mlogloss:1.38227	val-mlogloss:1.38227
[100]	train-mlogloss:1.17808	val-mlogloss:1.17847
[200]	train-mlogloss:1.12669	val-mlogloss:1.12765
[300]	train-mlogloss:1.11088	val-mlogloss:1.11255
[400]	train-mlogloss:1.10397	val-mlogloss:1.10650
[500]	train-mlogloss:1.09971	val-mlogloss:1.10320
[600]	train-mlogloss:1.09634	val-mlogloss:1.10086
[700]	train-mlogloss:1.09341	val-mlogloss:1.09903
[800]	train-mlogloss:1.09083	val-mlogloss:1.09751
[900]	train-mlogloss:1.08853	val-mlogloss:1.09623
[1000]	train-mlogloss:1.08639	val-mlogloss:1.09517
[1100]	train-mlogloss:1.08442	val-mlogloss:1.09425
[1200]	train-mlogloss:1.08256	val-mlogloss:1.09347
[1300]	train-mlogloss:1.08087	val-mlogloss:1.09280
[1400]	train-mlogloss:1.07927	val-mlogloss:1.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# XGBoost Tuning

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].map({"Class_1":0,"Class_2":1,"Class_3":2,"Class_4":3}).values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [ ]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial,data=X,target=y):
  """
  """
  param_space = { 
               'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
                'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.9),
                'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 0.9),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 0.9),
                'subsample': trial.suggest_float('subsample', 0.1, 0.9),
                'eta':trial.suggest_float('eta', 1e-2, 1e-2),
                'grow_policy': trial.suggest_categorical("grow_policy", ['depthwise','lossguide']),
                'max_depth': trial.suggest_int('max_depth',2,25),
                'seed': 0,
                'min_child_weight': trial.suggest_int('min_child_weight', 0, 300),
                'max_bin': trial.suggest_int('max_bin', 256, 512),
                'deterministic_histogram':trial.suggest_categorical('deterministic_histogram',[False]),
                "tree_method" : "gpu_hist",
                "predictor" : 'gpu_predictor',
                "objective" : 'multi:softprob',
                 "num_class":4
                }
            
  #X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.1,random_state=2021,stratify=y)
  k=5
  seed_list=[0]
  kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
  oof = np.zeros((len(df_all),4))
  score_list = []
  fold=1
  
  splits = list(kf.split(X,y))
  for train_idx, val_idx in splits:
    X_train, X_val = X[train_idx,:], X[val_idx,:]
    y_train, y_val = y[train_idx], y[val_idx]
  
    val_preds_list = []
  
    for seed in seed_list:
      # fit and run model
      param_space['seed'] = seed
      dtrain = xgb.DMatrix(data=X_train, label=y_train)
      dval = xgb.DMatrix(data=X_val, label=y_val)
      dtest = xgb.DMatrix(data=X_test)
      xgb.set_config(verbosity=0)

      
      model = xgb.train(param_space, dtrain,\
                       evals=[(dtrain,'train'),(dval,'val')],\
                       verbose_eval=False,
                       early_stopping_rounds=100,
                       num_boost_round=100000)
    
    

    
      val_preds_list.append(model.predict(dval))
     #test_preds_list.append(model.predict_proba(X_test)[:,1])
    
    oof[val_idx] = np.mean(val_preds_list,axis=0)
    score = log_loss(y_val, oof[val_idx])
    #print(f"fold: {fold},logloss: {score}")
    score_list.append(score)
    fold +=1
  
  cv_logloss = np.mean(score_list)
  
  return cv_logloss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-05-23 05:00:20,778] A new study created in memory with name: no-name-e60db3ba-ce7c-4f56-bd84-5f8d0c47fce7
[I 2021-05-23 05:08:05,253] Trial 0 finished with value: 1.0923741059530983 and parameters: {'lambda': 0.06661146995477027, 'alpha': 0.07848005248610329, 'colsample_bytree': 0.341013686587177, 'colsample_bynode': 0.5247363715016623, 'colsample_bylevel': 0.5698965984910002, 'subsample': 0.8495747203059043, 'eta': 0.01, 'grow_policy': 'depthwise', 'max_depth': 21, 'min_child_weight': 239, 'max_bin': 430, 'deterministic_histogram': False}. Best is trial 0 with value: 1.0923741059530983.
[I 2021-05-23 05:53:46,229] Trial 1 finished with value: 1.0912136118507059 and parameters: {'lambda': 0.15140190027219924, 'alpha': 1.7842160095709958, 'colsample_bytree': 0.30377579151729056, 'colsample_bynode': 0.5337136888830134, 'colsample_bylevel': 0.8068372064038368, 'subsample': 0.8130822433645674, 'eta': 0.01, 'grow_policy': 'lossguide', 'max_depth': 25, 'min_child_weight': 83, 'max_bi

Number of finished trials: 30
Best trial: {'lambda': 1.3718620937297796, 'alpha': 6.395781966352342, 'colsample_bytree': 0.2390564723786096, 'colsample_bynode': 0.7459555518737353, 'colsample_bylevel': 0.36002014547566097, 'subsample': 0.6302863949739616, 'eta': 0.01, 'grow_policy': 'lossguide', 'max_depth': 19, 'min_child_weight': 28, 'max_bin': 258, 'deterministic_histogram': False}


# Catboost

In [ ]:
# X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
# y = df_all['target'].map({"Class_1":0,"Class_2":1,"Class_3":2,"Class_4":3}).values
# X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
# random_seed = 0

# encoder = OrdinalEncoder()
# all_encoded = encoder.fit_transform(X.append(X_test))
# X = all_encoded[0:len(X)]
# X_test = all_encoded[len(X):]

# X = X.astype(int)
# X_test = X_test.astype(int)

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

X = X.astype(int)
X_test = X_test.astype(int)
cat_features = np.arange(0,X.shape[1]).tolist()

In [ ]:
# 'cat_features': cat_features,


# params = {'loss_function':'MultiClass',
#                   'eval_metric':'MultiClass', 
#                   'verbose': 500,
#           'depth': 6, 
#           'bootstrap_type': 'Bernoulli',
#           'l2_leaf_reg': 0.0006507380912214574, 
#           'random_strength': 8, 
#           'border_count': 55, 
#           'grow_policy': 'SymmetricTree', 
#           'learning_rate': 0.01, 
#           'min_data_in_leaf': 51, 
#           'subsample': 0.4635282300396178}

# params = {'learning_rate': 0.04, 
#           'depth': 5, 
#           'l2_leaf_reg': 993.0052814277057, 
#           'random_strength': 1.9065414654052146, 
#           'border_count': 128, 
#           'grow_policy': 'Depthwise', 
#           'min_data_in_leaf': 257}

params =   {'learning_rate': 0.03470328317940195, 
           'depth': 2, 
           'l2_leaf_reg': 820.7804346737378, 
           'random_strength': 0.336019499813798, 
           'border_count': 128,
           'grow_policy': 'Lossguide',
           'min_data_in_leaf': 267}


In [ ]:
params_cb = params

#params_cb["cat_features"] = cat_features
#params_cb ["learning_rate"] = 0.01
#params_cb ["depth"] = 4
params_cb ["loss_function"] = 'MultiClass'
params_cb ["od_wait"] = 1000
params_cb ["od_type"] = 'Iter'
#params_cb ["min_data_in_leaf"] = 1
#params_cb ["max_ctr_complexity"] = 15
params_cb ["task_type"] = "GPU"
params_cb["cat_features"] = cat_features



                      

name = 'catboost_3seeds_5fold'
k=5
seed_list=[0,1,2]
kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
oof = np.zeros((len(df_all),4))
test_preds_list = []
score_list = []
fold=1
  
splits = list(kf.split(X,y))
fold = 1
for train_idx, val_idx in splits:
  X_train, X_val = X[train_idx], X[val_idx]
  y_train, y_val = y[train_idx], y[val_idx]

  val_preds_list = []

  for seed in seed_list:
    
    # fit and run model
    params_cb['random_state'] = seed
        
    model = CatBoostClassifier(**params_cb,
            iterations=50000,
            use_best_model=True,
)

    model.fit(X_train,y=y_train,
              use_best_model=True,
              eval_set=[(X_val,y_val)],
              verbose=100)
    

    
    val_preds_list.append(model.predict_proba(X_val))
    test_preds_list.append(model.predict_proba(X_test))
    
  oof[val_idx] = np.mean(val_preds_list,axis=0)
  score = log_loss(y_val, oof[val_idx])
  print(f"fold: {fold},log_loss: {score}")
  score_list.append(score)
  # print(f"fold: {fold}, class0 tr %: {y_train.value_counts()[0]/len(y_train)}, class0 val %: {y_val.value_counts()[0]/len(y_val)} ")
  fold +=1
  
cv_logloss = np.mean(score_list)
print(f"{name} ,log_loss: {cv_logloss}")

preds= np.mean(test_preds_list,axis=0)


file_name_oof = name + "_oof.txt"
file_name_test = name + "_test.csv"
with open(file_name_oof, "wb") as fp:
      pickle.dump(oof, fp)

files.download(file_name_oof)

df_submission[['Class_1','Class_2','Class_3','Class_4']] = preds
df_submission.to_csv(file_name_test,index=None)
files.download(file_name_test) 

0:	learn: 1.3669102	test: 1.3669437	best: 1.3669437 (0)	total: 5.39ms	remaining: 4m 29s
100:	learn: 1.1115751	test: 1.1121498	best: 1.1121498 (100)	total: 441ms	remaining: 3m 37s
200:	learn: 1.1066347	test: 1.1076026	best: 1.1076026 (200)	total: 891ms	remaining: 3m 40s
300:	learn: 1.1034459	test: 1.1046238	best: 1.1046238 (300)	total: 1.36s	remaining: 3m 45s
400:	learn: 1.1010074	test: 1.1023501	best: 1.1023501 (400)	total: 1.9s	remaining: 3m 55s
500:	learn: 1.0990904	test: 1.1005899	best: 1.1005899 (500)	total: 2.45s	remaining: 4m 2s
600:	learn: 1.0974832	test: 1.0990813	best: 1.0990813 (600)	total: 3s	remaining: 4m 6s
700:	learn: 1.0961672	test: 1.0979376	best: 1.0979376 (700)	total: 3.52s	remaining: 4m 7s
800:	learn: 1.0950668	test: 1.0970042	best: 1.0970042 (800)	total: 4.06s	remaining: 4m 9s
900:	learn: 1.0940920	test: 1.0962171	best: 1.0962171 (900)	total: 4.6s	remaining: 4m 10s
1000:	learn: 1.0932931	test: 1.0955559	best: 1.0955559 (1000)	total: 5.14s	remaining: 4m 11s
1100:	lea

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# CBoost Tuning

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].map({"Class_1":0,"Class_2":1,"Class_3":2,"Class_4":3}).values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

X = X.astype(int)
X_test = X_test.astype(int)

In [ ]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 0

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

X = X.astype(int)
X_test = X_test.astype(int)
cat_features = np.arange(0,X.shape[1]).tolist()

In [ ]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial,data=X,target=y):
  """
  """
 

  param_space = {
        "od_type" : "Iter",
        "od_wait" : 1000,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 4e-2),
         "depth": trial.suggest_int("depth", 1, 16),
        "l2_leaf_reg": trial.suggest_loguniform('l2_leaf_reg', 1e-4, 1e3),
        "random_strength": trial.suggest_float("random_strength",0,3),
        # "bagging_temperature": trial.suggest_int("bagging_temperature",0,100),
        "border_count": trial.suggest_int("border_count",128,128),
        "grow_policy":trial.suggest_categorical("grow_policy",["Depthwise","SymmetricTree","Lossguide"]),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300)

        }
            
  #X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.1,random_state=2021,stratify=y)
  k=5
  seed_list=[0]
  kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
  oof = np.zeros((len(df_all),4))
  score_list = []
  fold=1
  
  splits = list(kf.split(X,y))
  for train_idx, val_idx in splits:
    X_train, X_val = X[train_idx,:], X[val_idx,:]
    y_train, y_val = y[train_idx], y[val_idx]

    if fold > 1:break

  
    val_preds_list = []
  
    
    for seed in seed_list:
      # fit and run model
      param_space['random_state'] = seed
      param_space ["loss_function"] = 'MultiClass'

      param_space["cat_features"] = cat_features

      model = CatBoostClassifier(**param_space,
                                task_type="GPU",
                                 iterations=50000,
                                 use_best_model=True)
      
      model.fit(X_train,y=y_train,
              embedding_features=None,
              use_best_model=True,
              eval_set=[(X_val,y_val)],
              verbose=0)
    
    
      val_preds_list.append(model.predict_proba(X_val))
     #test_preds_list.append(model.predict_proba(X_test)[:,1])
    
    oof[val_idx] = np.mean(val_preds_list,axis=0)
    score = log_loss(y_val, oof[val_idx])
    print(f"fold: {fold},logloss: {score}")
    score_list.append(score)
    fold +=1
  
  cv_logloss = np.mean(score_list)
  
  return cv_logloss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-05-26 01:14:24,115] A new study created in memory with name: no-name-3ad323a3-ad18-49b1-8059-860d69aafae9
[I 2021-05-26 01:17:18,543] Trial 0 finished with value: 1.0936730625857125 and parameters: {'learning_rate': 0.013156916090252357, 'depth': 11, 'l2_leaf_reg': 0.5384381111955518, 'random_strength': 1.2907674309630877, 'border_count': 128, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 102}. Best is trial 0 with value: 1.0936730625857125.


fold: 1,logloss: 1.0936730625857125


[I 2021-05-26 01:17:56,075] Trial 1 finished with value: 1.0917748757104253 and parameters: {'learning_rate': 0.03371656914750022, 'depth': 15, 'l2_leaf_reg': 0.002015270203057493, 'random_strength': 2.7869012982784773, 'border_count': 128, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 251}. Best is trial 1 with value: 1.0917748757104253.


fold: 1,logloss: 1.0917748757104253


[I 2021-05-26 01:18:31,408] Trial 2 finished with value: 1.09123335957795 and parameters: {'learning_rate': 0.029761539054575115, 'depth': 13, 'l2_leaf_reg': 3.870654880389206, 'random_strength': 2.846634602474385, 'border_count': 128, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 174}. Best is trial 2 with value: 1.09123335957795.


...

fold: 1,logloss: 1.0899209516737272


[I 2021-05-26 03:27:57,846] Trial 98 finished with value: 1.0892932681969183 and parameters: {'learning_rate': 0.030297504795236803, 'depth': 3, 'l2_leaf_reg': 736.3557840598204, 'random_strength': 0.828589442581315, 'border_count': 128, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 258}. Best is trial 61 with value: 1.0892181195137285.


fold: 1,logloss: 1.0892932681969183


[I 2021-05-26 03:28:59,054] Trial 99 finished with value: 1.0893328322289726 and parameters: {'learning_rate': 0.03052707157167038, 'depth': 3, 'l2_leaf_reg': 776.6442381094598, 'random_strength': 0.7784514824031952, 'border_count': 128, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 257}. Best is trial 61 with value: 1.0892181195137285.


fold: 1,logloss: 1.0893328322289726
Number of finished trials: 100
Best trial: {'learning_rate': 0.03470328317940195, 'depth': 2, 'l2_leaf_reg': 820.7804346737378, 'random_strength': 0.336019499813798, 'border_count': 128, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 267}
